In [2]:
#importing ncessary libraries
import torch
from transformers import GPTJForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import pandas as pd

In [3]:
model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", torch_dtype=torch.float16,low_cpu_mem_usage=True)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [4]:

def get_summary_generated(text):
    """
    Function to generate the summary of the text
    """
    input_ids = tokenizer.encode(text, truncation=True, max_length=model.config.max_position_embeddings, return_tensors="pt").to(device)

    input_length = input_ids.size(1)
    max_length = min(int(input_length * 1.5), model.config.max_position_embeddings)

    summary_ids = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.9,
        # num_beams=4,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summary = summary.replace(text, "").strip()

    return summary



In [6]:
# Reading the data file
data= pd.read_csv("text_data.csv")
data["summarized_text"] = ""
for index, row in data.iterrows():
    original_text = row["original_text"]
    summary = get_summary_generated(original_text)
    data.at[index, "summarized_text"] = summary

In [7]:
data

,original_text,summarized_text
0,America has changed dramatically during recent...,Retrieved from http://faculty.mit.edu/egf/chan...
1,So how do you go about identifying your streng...,SWOT Analysis is a useful technique that helps...
2,Jupiter is the fifth planet from the Sun and t...,Jupiter's gravitational field is weaker than t...


In [ ]:
data.to_csv("processed_text_summar"